# Test the model on a video or live feed

### Load necessary libraries

In [1]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import model_from_json

### Some pre-processing

In [2]:
def get_key(val):
    for key, value in pred_dict.items():
         if val == value:
             return key
 
    return "key doesn't exist"

pred_dict = {'defect_1':0,'defect_2':1,'normal':2}

### Load the json model file

In [3]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


### Run the video and apply the model

In [4]:
cap = cv2.VideoCapture(r'test.mp4')
if (cap.isOpened()== False):
  print("Error opening video stream or file")

while(cap.isOpened()):
  ret, frame = cap.read()

  if ret == True:
    frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1.2, 100)
    
    if circles is not None:
      circles = np.round(circles[0, :]).astype("int")
      for (x, y, r) in circles:
        cv2.circle(frame, (x, y), r, (0, 255, 0), 4)
      
      copy_frame = frame[y-r:y+r,x-r:x+r]
      img = cv2.resize(copy_frame,(168,168))

      X =  image.img_to_array(img)
      X = np.expand_dims(X,axis=0)
      images = np.vstack([X])
      val = loaded_model.predict_classes(images)
      string = get_key(val[0])
      # Display the resulting frame
      font = cv2.FONT_HERSHEY_DUPLEX
      try:
        cv2.putText(frame, string, (x-r, y-r), font, 1.0, (255, 255, 255), 1)
        cv2.imshow('Test video',frame)        
      except:
        pass
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
 
  else:
    break
 
cap.release()
 
cv2.destroyAllWindows()

C:\Users\Het\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
